In [1]:
!pip install -U --quiet langchain-google-genai  langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.3 MB/s eta 0:00:00


In [3]:
import getpass
import os

# Set Google AI Key
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

Provide your Google API Key··········


# Simple Example

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [6]:
# setup the gemini pro
gemini_llm = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.3)

In [7]:
result = gemini_llm.invoke("what is google vertex ai?")
print(result.content)

Google Vertex AI is a managed machine learning platform that provides a unified experience for developing, deploying, and managing machine learning models. It offers a wide range of tools and services to help users build and deploy models quickly and easily, including:

* **AutoML:** A set of pre-built machine learning models that can be used for common tasks such as image classification, object detection, and natural language processing.
* **BigQuery ML:** A library of machine learning algorithms that can be used to train models on data stored in BigQuery.
* **Cloud AI Platform Notebooks:** A cloud-based Jupyter notebook environment for developing and experimenting with machine learning models.
* **Cloud AI Platform Pipelines:** A service for creating and managing machine learning pipelines.
* **Cloud AI Platform Prediction:** A service for deploying and serving machine learning models.
* **Cloud AI Platform Training:** A service for training machine learning models.

Vertex AI is des

# Example With Prompt Template


In [10]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import HumanMessagePromptTemplate

**Gemini doesn’t support SystemMessage at the moment, but it can be added to the first human message in the row.**

In [11]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "return the sentiment for the given text in json fromat the sentiment value can be 'nagative','positive'"
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)


In [12]:
chat_message =  chat_template.format_messages(text="i don' like weather today.")

In [14]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3, convert_system_message_to_human=True) # set the convert_system_message_to_human to true

In [16]:
llm.invoke(chat_message)

AIMessage(content='```json\n{\n  "sentiment": "negative"\n}\n```')

# Use Output parser to get formated output

In [17]:
from langchain_core.output_parsers import JsonOutputParser

In [18]:
parser = JsonOutputParser()

In [19]:
chain =  llm | parser

In [20]:
chat_message

[SystemMessage(content="return the sentiment for the given text in json fromat the sentiment value can be 'nagative','positive'"),
 HumanMessage(content="i don' like weather today.")]

In [22]:
sentiment = chain.invoke(chat_message)
print(sentiment)

{'sentiment': 'negative'}


# Tagging Chain with Gemini Pro

In [31]:
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator

In [24]:
class Tags(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

In [32]:
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Tags)

In [33]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [34]:
chain = prompt | llm | parser

chain.invoke({"query": "i don' like weather today."})

Tags(sentiment='sad', aggressiveness=2, language='english')

# Create a Conversational chain with Gemini pro

In [37]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [38]:
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

In [39]:
conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


'Hello! How may I assist you today?'

In [40]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello! How may I assist you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


"That's great to hear! I'm glad you're having a good time. I'm always happy to chat with people and learn new things. What do you like to talk about?"

Adding a coustom prompt to the conversationchain

In [45]:
PROMPT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI Assistant:
"""

PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=PROMPT_TEMPLATE
)

In [46]:
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    prompt=PROMPT,
    memory=ConversationBufferMemory(ai_prefix="AI Assistant")
)

In [47]:
conversation.predict(input="who is Gojo?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: who is Gojo?
AI Assistant:


> Finished chain.


'Gojo Satoru is a character in the popular manga and anime series Jujutsu Kaisen. He is a powerful jujutsu sorcerer and a teacher at Tokyo Jujutsu High. Gojo is known for his eccentric personality, his immense strength, and his unique appearance, which includes white hair and blue eyes. He is also a skilled swordsman and hand-to-hand combatant.\n\nGojo is a complex and enigmatic character. He is often portrayed as aloof and carefree, but he also has a deep sense of justice and compassion. He is fiercely loyal to his students and friends, and he is always willing to fight for what he believes in.\n\nGojo is one of the most popular characters in Jujutsu Kaisen, and he has been praised for his unique design, his powerful abilities, and his complex personality. He is a fan favorite, and he is sure to continue to be a popular character for years to come.'

In [48]:
conversation.predict(input="What is his power?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: who is Gojo?
AI Assistant: Gojo Satoru is a character in the popular manga and anime series Jujutsu Kaisen. He is a powerful jujutsu sorcerer and a teacher at Tokyo Jujutsu High. Gojo is known for his eccentric personality, his immense strength, and his unique appearance, which includes white hair and blue eyes. He is also a skilled swordsman and hand-to-hand combatant.

Gojo is a complex and enigmatic character. He is often portrayed as aloof and carefree, but he also has a deep sense of justice and compassion. He is fiercely loyal to his students and friends, and he is always willing to fight for what he believes in.

Gojo is one of the most popular characters

'Gojo Satoru\'s power is known as "Limitless," and it is one of the strongest techniques in the Jujutsu Kaisen universe. Limitless allows Gojo to manipulate space and time around him, creating a variety of powerful effects.\n\nOne of the most notable aspects of Limitless is its ability to create an "Infinity" barrier around Gojo. This barrier is impenetrable, and it prevents any attacks from reaching him. Gojo can also use Limitless to teleport himself and others, and he can even use it to stop time for a brief period.\n\nIn addition to its defensive capabilities, Limitless also grants Gojo a number of offensive abilities. He can use it to fire powerful blasts of energy, and he can also create powerful shockwaves by clapping his hands. Gojo is also a skilled swordsman, and he can use Limitless to enhance his swordsmanship.\n\nOverall, Limitless is an incredibly versatile and powerful technique that makes Gojo one of the strongest characters in Jujutsu Kaisen.'

In [49]:
conversation.predict(input="what is his clan name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: who is Gojo?
AI Assistant: Gojo Satoru is a character in the popular manga and anime series Jujutsu Kaisen. He is a powerful jujutsu sorcerer and a teacher at Tokyo Jujutsu High. Gojo is known for his eccentric personality, his immense strength, and his unique appearance, which includes white hair and blue eyes. He is also a skilled swordsman and hand-to-hand combatant.

Gojo is a complex and enigmatic character. He is often portrayed as aloof and carefree, but he also has a deep sense of justice and compassion. He is fiercely loyal to his students and friends, and he is always willing to fight for what he believes in.

Gojo is one of the most popular characters

'Gojo Satoru does not belong to any clan in the Jujutsu Kaisen universe. He is a member of the Gojo family, but the Gojo family is not considered a clan in the traditional sense. The Gojo family is a group of powerful jujutsu sorcerers, but they do not have a formal structure or hierarchy.\n\nGojo Satoru is the head of the Gojo family, and he is considered to be one of the strongest jujutsu sorcerers in the world. He is also a teacher at Tokyo Jujutsu High, where he trains the next generation of jujutsu sorcerers.'